# Build a Customer Service Agent using Granite-3.1-8B-Instruct model and LangChain Framework

LLMs possess powerful language understanding and reasoning capabilities. To develop a customer service chatbot or agent, an LLM must be able to interact with external data systems, such as databases, to retrieve real-time order statuses or update customer information.

IBM’s Granite-3.1-8B-Instruct is a compact model with robust function-calling capabilities, making it suitable for building agents. This model enables the LLM to interpret customer queries, interact with APIs, and perform real-time data operations on a database through function-calling, ensuring information is accurately updated and maintained.

## Environment Setup

### Install dependencies

In [ ]:
! pip install git+https://github.com/ibm-granite-community/utils "langchain_community<0.3.0" replicate pydantic pandas

In [2]:
# imports
import pandas as pd
import sqlite3
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

### Instantiate the model object with Granite-3.0-8B-Instruct Model

In [3]:
model = Replicate(
    model="ibm-granite/granite-3.1-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"temperature": 0}
)

In [ ]:
model.invoke("What is the meaning of life")

## Create Database

To demonstrate an example of a customer service agent interacting with databases via API, we created synthetic data. Using this data, we set up two SQLite databases: customers and orders.

In [5]:
# Synthetic Data 
# Define the data as a list of dictionaries
customer_data = [
    {"customer_id": 1, "name": "Alice Smith", "email": "alice@example.com", "phone": "555-1234"},
    {"customer_id": 2, "name": "Bob Johnson", "email": "bob@example.com", "phone": "555-5678"},
    {"customer_id": 3, "name": "Charlie Lee", "email": "charlie@example.com", "phone": "555-8765"},
    {"customer_id": 4, "name": "Daisy White", "email": "daisy@example.com", "phone": "555-2345"},
    {"customer_id": 5, "name": "Ethan Clark", "email": "ethan@example.com", "phone": "555-6789"}
]
order_data = [
    {"order_id": 1, "customer_id": 1, "product_name": "Laptop", "order_status": "Shipped", "order_date": "2024-09-20"},
    {"order_id": 2, "customer_id": 2, "product_name": "Smartphone", "order_status": "Delivered", "order_date": "2024-09-18"},
    {"order_id": 3, "customer_id": 3, "product_name": "Headphones", "order_status": "Processing", "order_date": "2024-09-22"},
    {"order_id": 4, "customer_id": 4, "product_name": "Monitor", "order_status": "Cancelled", "order_date": "2024-09-17"},
    {"order_id": 5, "customer_id": 5, "product_name": "Keyboard", "order_status": "Delivered", "order_date": "2024-09-16"}
]

df_customer = pd.DataFrame(customer_data)
df_order = pd.DataFrame(order_data)

In [ ]:
conn = sqlite3.connect('example.db')

# Write the DataFrame to the SQLite database
df_customer.to_sql('customers', conn, if_exists='replace', index=False)
df_order.to_sql('orders', conn, if_exists='replace', index=False)

# Close the connection
# conn.close()

### Query the database to verify the data

In [ ]:
df = pd.read_sql_query("SELECT * FROM orders", conn)
df

In [ ]:
df = pd.read_sql_query("SELECT * FROM customers", conn)
df

## Develop tools to interact with the database

Creating tools for function calling in LLMs involves designing specialized functions that facilitate interaction with external systems, such as databases, APIs, and other applications. By building robust, well-defined function-calling tools, developers enable LLMs to perform specific tasks with precision, like accessing customer data or executing transactions. This approach enhances the LLM’s capability to respond to complex user queries and handle dynamic workflows efficiently.

We created two tools: <br>
**get_order_by_order_id**: Retrieves the order status based on a given order ID from the orders table. <br>
**edit_customer_email_by_customer_name**: Updates a customer's email based on their name in the customers table. 

In [9]:
@tool
def get_order_by_order_id(order_id: int):
    """
    Retrieve an order_status given the order_id from the orders table.

    Parameters:
    order_id (int): The ID of the order to retrieve.
    """
    df = pd.read_sql_query("SELECT product_name,order_status FROM orders where order_id= {0}".format(order_id), conn)
    if not df.empty:
        return df.iloc[0]
    else:
        return None

In [10]:
@tool
def edit_customer_email_by_customer_name(customer_name:str, new_email:str):
    """
    Update the email of a customer given their name in the customer table.

    Parameters:
    customer_name (str): The name of the customer whose email needs to be updated.
    new_email (str): The new email address to set.
    """
    cursor = conn.cursor()
    update_query = "UPDATE customers SET email = '{0}' WHERE name = '{1}'".format(new_email, customer_name)
    cursor.execute(update_query)
    conn.commit() # Commit the changes to the database
    df = pd.read_sql_query("SELECT email FROM customers where name= '{0}'".format(customer_name), conn)
    return df

In [11]:
tools = [get_order_by_order_id, edit_customer_email_by_customer_name]

## Build the LLM agent using the designated tools

### Establish the prompt template

We will set up a new prompt template to ask multiple questions. This template is more complex. It is referred to as a [structured chat prompt](https://api.python.langchain.com/en/latest/agents/langchain.agents.structured_chat.base.create_structured_chat_agent.html#langchain-agents-structured-chat-base-create-structured-chat-agent) and can be used for creating agents that have multiple tools available. In our case, the tool we are using were defined in previous step. The structured chat prompt will be made up of a `system_prompt`, a `human_prompt` and our `tools`. 

First, we will set up the `system_prompt`. This prompt instructs the agent to print its "thought process," which involves the agent's subtasks, the tools that were used and the final output. This gives us insight into the agent's function calling. The prompt also instructs the agent to return its responses in JSON Blob format.

In [12]:
system_prompt = """You are a helpful assistant. Use the following tools to answer queries:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```
Always use tools when possible to gather information before answering.
If you can't answer based on the tool outputs and given context, say so.
Keep your answers concise and based solely on the provided information."""

In the following code, we are establishing the `human_prompt`. This prompt tells the agent to display the user input followed by the intermediate steps taken by the agent as part of the `agent_scratchpad`.

In [13]:
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

Next, we establish the order of our newly defined prompts in the prompt template. We create this new template to feature the `system_prompt` followed by an optional list of messages collected in the agent's memory, if any, and finally, the `human_prompt` which includes both the human input and `agent_scratchpad`.

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

Now, let's finalize our prompt template by adding the tool names, descriptions and arguments using a [partial prompt template](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/partial/). This allows the agent to access the information pertaining to each tool including the intended use cases and also means we can add and remove tools without altering our entire prompt template.

In [15]:
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

An important feature of AI agents is their memory. Agents are able to store past conversations and past findings in their memory to improve the accuracy and relevance of their responses going forward. In our case, we will use LangChain's `ConversationBufferMemory()` as a means of memory storage. 

In [16]:
memory = ConversationBufferMemory()

And now we can set up a chain with our agent's scratchpad, memory, prompt and the LLM. The AgentExecutor class is used to execute the agent. It takes the agent, its tools, error handling approach, verbose parameter and memory as parameters.

In [17]:
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | model
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=False, memory=memory
)

## Generate responses using the customer service agent

### Request the order status from the database

In [ ]:
memory.clear()
agent_executor.invoke({"input": "What is the status of customer's order 4?"})

### Request to update a customer's email address

In [ ]:

memory.clear()
agent_executor.invoke({"input": "Update email id of Daisy white to daisywhite.india@gmail.com"})

#### Verify that the database has been updated with the new email address

In [ ]:
df = pd.read_sql_query("SELECT * FROM customers", conn)
df

#### Close the database connection

In [21]:
# Close the connection
conn.close()